In [14]:
import cv2
import os
import pandas as pd
import numpy as np

In [15]:
def parse_file(path_file):
    result = []

    with open(path_file, 'r') as f:
        for line in f.readlines():
            data = line.split('\n')[0].split(' ')
            result.append([int(data[0]), int(data[1]), int(data[2])])
            assert result[-1][0] <= result[-1][1], result[-1]
            if len(result) > 1:
                assert result[-1][0] == result[-2][1], (result[-1], result[-2])
    return result

In [21]:
MAIN_DIR = './data/6/'
VIDEO_PATH = os.path.join(MAIN_DIR, 'video.mp4')
LABELS_PATH = os.path.join(MAIN_DIR, 'labels.txt')
OUT_PATH = os.path.join(MAIN_DIR, 'img/')
DESCRIPTION_PATH = os.path.join(MAIN_DIR, 'description.csv')
ANNOTATION_PATH = os.path.join(MAIN_DIR, '_annotation.csv')
VIDEO_LABELS_PATH = os.path.join(MAIN_DIR + 'labels.txt')

In [22]:
train_capture = cv2.VideoCapture(VIDEO_PATH)
assert train_capture.isOpened()

In [23]:
SECONDS_PER_FRAME = 1
FPS = int(train_capture.get(cv2.CAP_PROP_FPS))
DURATION = int(train_capture.get(cv2.CAP_PROP_FRAME_COUNT)) // FPS

print('FPS: ', FPS, ', duration: ', DURATION, sep='')

FPS: 30, duration: 120


In [24]:
cv2.EVENT_FLAG_SHIFTKEY
img = np.zeros((512,512,3), np.uint8)


In [25]:
drawing = False
ix,iy = -1,-1
frame = None
current_class = -1
flag = False
current_coordinates = np.array([[0,0], [0,0]])
row = []

def draw_reactangle_with_drag(event, x, y, flags, param):
    global ix, iy, drawing, frame, current_class, current_coordinates
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix = x
        iy = y


    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            train_capture.set(cv2.CAP_PROP_POS_MSEC, current_video_position * 1000)
            _, img_tmp = train_capture.read()
            cv2.rectangle(img_tmp, pt1=(ix,iy), pt2=(x, y),color=(0,255,0),thickness=4)
            frame = img_tmp

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        train_capture.set(cv2.CAP_PROP_POS_MSEC, current_video_position * 1000)
        _, img_tmp = train_capture.read()
        cv2.rectangle(img_tmp, pt1=(ix,iy), pt2=(x, y),color=(0,255,0),thickness=4)
        current_coordinates = np.array([[ix,iy], [x,y]])
        current_class = 1
        frame = img_tmp
    elif event == cv2.EVENT_FLAG_RBUTTON:
        current_class = 0
        current_coordinates = np.array([[0,0], [0,0]])

In [26]:
previous_class = None
current_start = 0
current_picture_idx = 1
video_labels = []
image_labels = {}

for current_video_position in range(0, DURATION + SECONDS_PER_FRAME, SECONDS_PER_FRAME):
    train_capture.set(cv2.CAP_PROP_POS_MSEC, current_video_position * 1000)
    ret, frame = train_capture.read()
    if not ret:
        print('End of stream')
        break
    cv2.namedWindow('Frame')
    cv2.setMouseCallback('Frame',draw_reactangle_with_drag)
    image_name = str(current_picture_idx) + '.jpg'
    cv2.imwrite(os.path.join(OUT_PATH, image_name), frame)
    while(current_class == -1):
        cv2.imshow('Frame',frame)
        k = cv2.waitKey(1) & 0xFF    
    image_labels[image_name] = current_class
    if previous_class is None:
        previous_class = current_class
        current_start = 0
    elif previous_class != current_class:
        video_labels.append([current_start, current_video_position, previous_class])
        current_start = current_video_position
    row.append({'file': image_name, 'width': w, 'height': h,'class': current_class, 
                'xmin': np.min(current_coordinates[:,0]), 'ymin': np.min(current_coordinates[:,1]),
                'xmax': np.max(current_coordinates[:,0]), 'ymax': np.max(current_coordinates[:,1])})
    previous_class = current_class
    current_picture_idx += 1
    current_class = -1

        
video_labels.append([current_start, DURATION, previous_class])

train_capture.release()
cv2.destroyAllWindows()

In [13]:
pd.Series(image_labels).to_csv(DESCRIPTION_PATH, header=['class_id'], index_label='image_name')
pd.DataFrame(video_labels).set_index(0).to_csv(VIDEO_LABELS_PATH, sep=' ', header=False)
pd.DataFrame(row).set_index('file').to_csv(ANNOTATION_PATH)

439